# XGBoost Deep Dive: От теории к практике

## 🎯 Цели ноутбука

1. **Глубокое понимание математики** градиентного бустинга и XGBoost
2. **Практическое применение** на реальной бизнес-задаче кредитного скоринга
3. **Детальный анализ** гиперпараметров и их влияния
4. **Best practices** для использования XGBoost в production

## 💼 Бизнес-задача: Кредитный скоринг

**Контекст:** Крупный тайваньский банк выдает кредитные карты клиентам. Необходимо предсказать, кто из клиентов сделает дефолт (не заплатит) в следующем месяце.

**Почему это важно:**
- 💰 **Финансовые потери:** Дефолт = потеря денег банком
- ⚖️ **Регуляторные требования:** Basel III требует точной оценки рисков
- 🎯 **Оптимизация:** Баланс между одобрением заявок и минимизацией рисков
- 📊 **Интерпретируемость:** Необходимо объяснить решение клиенту

**Метрики успеха:**
- **Precision:** Какой % из предсказанных дефолтов действительно дефолты (минимизация false positives)
- **Recall:** Какой % реальных дефолтов мы поймали (минимизация false negatives)
- **ROC-AUC:** Общее качество ранжирования
- **PR-AUC:** Важнее для несбалансированных данных

**Стоимость ошибок:**
- **False Negative (пропустили дефолт):** Банк теряет деньги (~5000-50000 TWD)
- **False Positive (отказали хорошему клиенту):** Потеря прибыли от процентов (~500-2000 TWD)
- Соотношение: FN в 10-100 раз дороже FP

---

## 📚 Часть 1: Теоретический фундамент

### 1.1 От решающих деревьев к градиентному бустингу

#### Решающее дерево (Decision Tree)

Базовый строительный блок. Дерево разделяет пространство признаков на регионы.

**Математически:** Предсказание дерева $T(x; \Theta)$, где $\Theta = \{R_j, \gamma_j\}_{j=1}^J$

$$f(x) = \sum_{j=1}^{J} \gamma_j \cdot \mathbb{1}(x \in R_j)$$

где:
- $J$ - количество листьев (терминальных узлов)
- $R_j$ - регион (прямоугольная область в пространстве признаков)
- $\gamma_j$ - предсказание для региона $R_j$
- $\mathbb{1}(\cdot)$ - индикаторная функция

**Алгоритм построения (жадный, рекурсивный):**

1. Начинаем с корня (все данные)
2. Для каждого признака $k$ и порога $s$:
   - Разбиваем на $R_L(k,s) = \{X | X_k ≤ s\}$ и $R_R(k,s) = \{X | X_k > s\}$
3. Выбираем разбиение, минимизирующее loss:

$$\min_{k,s} \left[ \min_{\gamma_L} \sum_{x_i \in R_L} L(y_i, \gamma_L) + \min_{\gamma_R} \sum_{x_i \in R_R} L(y_i, \gamma_R) \right]$$

4. Рекурсивно повторяем для дочерних узлов

**Проблемы одного дерева:**
- 🔴 **Высокая дисперсия:** Маленькое изменение данных → большое изменение дерева
- 🔴 **Переобучение:** Слишком подстраивается под обучающую выборку
- 🔴 **Жесткие границы:** Не smooth предсказания

---

### 1.2 Бэггинг (Bootstrap Aggregating)

**Идея:** Усреднение уменьшает дисперсию!

$$f_{bag}(x) = \frac{1}{B} \sum_{b=1}^{B} f_b(x)$$

где $f_b$ - дерево, обученное на bootstrap выборке $b$.

**Почему работает:**

Пусть $f_1, \ldots, f_B$ - независимые случайные величины с дисперсией $\sigma^2$. Тогда:

$$\text{Var}\left(\frac{1}{B} \sum_{b=1}^B f_b\right) = \frac{\sigma^2}{B}$$

Дисперсия уменьшается в $B$ раз!

**Random Forest = Bagging + Feature randomness** (для увеличения разнообразия)

---

### 1.3 Бустинг: Последовательное обучение

**Идея бустинга:** Вместо независимых моделей, обучаем последовательно, каждая следующая модель исправляет ошибки предыдущих.

#### AdaBoost (Adaptive Boosting) - исторический контекст

Переваживание примеров:

$$w_i^{(t+1)} = w_i^{(t)} \cdot \exp(\alpha_t \cdot \mathbb{1}(y_i \neq f_t(x_i)))$$

Финальное предсказание:

$$F(x) = \text{sign}\left(\sum_{t=1}^T \alpha_t f_t(x)\right)$$

**Ограничение:** Работает только для классификации, не обобщается на другие loss функции.

---

### 1.4 Градиентный бустинг: Универсальный подход

#### Ключевая идея (Jerome Friedman, 2001)

Бустинг = **Оптимизация в функциональном пространстве**

**Постановка задачи:**

Хотим найти функцию $F^*(x)$, минимизирующую ожидаемые потери:

$$F^* = \arg\min_F \mathbb{E}_{x,y}[L(y, F(x))]$$

где $L(y, F(x))$ - функция потерь (loss function).

**Аппроксимация на конечной выборке:**

$$F^* \approx \arg\min_F \sum_{i=1}^n L(y_i, F(x_i))$$

#### Алгоритм Gradient Boosting

**Шаг 0 (Инициализация):**

$$F_0(x) = \arg\min_\gamma \sum_{i=1}^n L(y_i, \gamma)$$

Для regression (MSE): $F_0(x) = \bar{y}$ (среднее)  
Для classification (log-loss): $F_0(x) = \log\frac{p}{1-p}$ (log-odds)

**Шаг $m = 1, 2, \ldots, M$:**

1. **Вычисляем псевдо-остатки (negative gradient):**

$$r_{im} = -\left[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right]_{F=F_{m-1}}$$

   Интуиция: $r_{im}$ показывает направление, в котором нужно "двигаться", чтобы уменьшить loss для примера $i$.

2. **Обучаем дерево $h_m(x)$ на предсказание остатков:**

$$h_m = \arg\min_h \sum_{i=1}^n (r_{im} - h(x_i))^2$$

   Дерево аппроксимирует anti-gradient!

3. **Для каждого листа $j$ дерева $h_m$, оптимизируем значение:**

$$\gamma_{jm} = \arg\min_\gamma \sum_{x_i \in R_{jm}} L(y_i, F_{m-1}(x_i) + \gamma)$$

   Line search в направлении gradient для каждого региона.

4. **Обновляем модель:**

$$F_m(x) = F_{m-1}(x) + \nu \cdot h_m(x)$$

   где $\nu$ - learning rate (shrinkage parameter), обычно $\nu \in [0.01, 0.3]$

**Финальная модель:**

$$F(x) = F_0(x) + \nu \sum_{m=1}^M h_m(x)$$

---

#### Примеры loss функций и их градиентов

**1. Regression (MSE):**

$$L(y, F) = \frac{1}{2}(y - F)^2$$

$$-\frac{\partial L}{\partial F} = y - F = \text{остаток}$$

Градиентный бустинг для MSE = обычный бустинг по остаткам!

**2. Binary Classification (Log-Loss):**

$$L(y, F) = \log(1 + e^{-2yF}), \quad y \in \{-1, +1\}$$

$$-\frac{\partial L}{\partial F} = \frac{2y}{1 + e^{2yF}} = 2y \cdot p_{model}(y|x)$$

где $p_{model}(y|x) = \frac{1}{1 + e^{-2yF}}$

**3. Multi-class (Softmax):**

Обучаем $K$ деревьев на каждой итерации (одно для каждого класса).

---

### 1.5 XGBoost: eXtreme Gradient Boosting

#### Ключевые инновации (Tianqi Chen, 2016)

**1. Second-Order Taylor Expansion (2nd order gradient)**

Стандартный GB использует только первый градиент. XGBoost использует разложение Тейлора 2го порядка:

$$L(y, F_{m-1} + h_m) \approx L(y, F_{m-1}) + g_i \cdot h_m(x_i) + \frac{1}{2} h_i \cdot h_m^2(x_i)$$

где:
- $g_i = \frac{\partial L(y_i, F)}{\partial F}\Big|_{F=F_{m-1}}$ - **первый градиент** (gradient)
- $h_i = \frac{\partial^2 L(y_i, F)}{\partial F^2}\Big|_{F=F_{m-1}}$ - **второй градиент** (hessian)

**Зачем нужен hessian:**
- 📊 **Лучшая аппроксимация** loss функции (квадратичная vs линейная)
- 🎯 **Автоматическая нормализация** (второй градиент = мера "уверенности")
- ⚡ **Быстрее сходится** (как метод Ньютона vs градиентный спуск)

**Пример для MSE:**
- $g_i = F_{m-1}(x_i) - y_i$ (предсказание - истина)
- $h_i = 1$ (константа)

**Пример для Log-Loss:**
- $g_i = p_i - y_i$ где $p_i = \sigma(F_{m-1}(x_i))$
- $h_i = p_i(1 - p_i)$ (дисперсия Бернулли)

---

**2. Регуляризованная целевая функция**

XGBoost минимизирует:

$$\mathcal{L}^{(m)} = \sum_{i=1}^n L(y_i, F_{m-1}(x_i) + h_m(x_i)) + \Omega(h_m)$$

где регуляризация:

$$\Omega(h) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T w_j^2$$

- $T$ - количество листьев дерева
- $w_j$ - значение (вес) в листе $j$
- $\gamma$ - штраф за сложность (количество листьев)
- $\lambda$ - L2 регуляризация на веса

**Интуиция:**
- $\gamma T$ - «Occam's Razor»: проще деревья лучше
- $\lambda ||w||^2$ - маленькие веса → более smooth предсказания

---

**3. Оптимальное значение листа**

Подставляем разложение Тейлора и регуляризацию:

$$\mathcal{L}^{(m)} \approx \sum_{i=1}^n \left[g_i h_m(x_i) + \frac{1}{2}h_i h_m^2(x_i)\right] + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T w_j^2$$

Группируем примеры по листьям. Пусть $I_j = \{i | x_i \in \text{лист } j\}$:

$$\mathcal{L}^{(m)} = \sum_{j=1}^T \left[\left(\sum_{i \in I_j} g_i\right) w_j + \frac{1}{2}\left(\sum_{i \in I_j} h_i + \lambda\right) w_j^2\right] + \gamma T$$

Обозначим:
- $G_j = \sum_{i \in I_j} g_i$ - сумма градиентов в листе $j$
- $H_j = \sum_{i \in I_j} h_i$ - сумма вторых градиентов в листе $j$

Тогда:

$$\mathcal{L}^{(m)} = \sum_{j=1}^T \left[G_j w_j + \frac{1}{2}(H_j + \lambda) w_j^2\right] + \gamma T$$

Это квадратичная функция от $w_j$! Минимизируем по $w_j$:

$$\frac{\partial \mathcal{L}^{(m)}}{\partial w_j} = G_j + (H_j + \lambda)w_j = 0$$

**Оптимальный вес листа:**

$$w_j^* = -\frac{G_j}{H_j + \lambda}$$

**Минимальная loss для фиксированной структуры дерева:**

$$\mathcal{L}^{(m)*} = -\frac{1}{2}\sum_{j=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma T$$

---

**4. Алгоритм построения дерева (Split Finding)**

**Цель:** Найти оптимальное разбиение для минимизации loss.

Для каждого признака и порога вычисляем **gain** (уменьшение loss):

$$\text{Gain} = \frac{1}{2}\left[\frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L + G_R)^2}{H_L + H_R + \lambda}\right] - \gamma$$

где:
- $G_L, H_L$ - сумма градиентов/hessians в левом поддереве
- $G_R, H_R$ - в правом поддереве
- $\gamma$ - штраф за добавление нового разбиения

**Интерпретация Gain:**

1. **Первый термин:** Gain от разделения на два узла
2. **Второй термин:** Loss если бы не разделяли (один узел)
3. **Третий термин:** Штраф за сложность

**Критерий остановки:** Если $\text{Gain} < 0$, разбиение не улучшает модель.

**Алгоритм (псевдокод):**

```
def BuildTree(градиенты g, hessians h, глубина):
    if глубина > max_depth or len(g) < min_samples:
        return Лист с весом w = -sum(g) / (sum(h) + lambda)
    
    best_gain = 0
    best_split = None
    
    # Перебор всех признаков
    for признак in признаки:
        # Перебор всех порогов
        for порог in уникальные_значения(признак):
            G_L = sum(g[признак <= порог])
            H_L = sum(h[признак <= порог])
            G_R = sum(g[признак > порог])
            H_R = sum(h[признак > порог])
            
            gain = 0.5 * (G_L^2/(H_L + lambda) + G_R^2/(H_R + lambda) 
                         - (G_L + G_R)^2/(H_L + H_R + lambda)) - gamma
            
            if gain > best_gain:
                best_gain = gain
                best_split = (признак, порог)
    
    if best_gain == 0:
        return Лист с весом w = -sum(g) / (sum(h) + lambda)
    
    левое_поддерево = BuildTree(g_left, h_left, глубина+1)
    правое_поддерево = BuildTree(g_right, h_right, глубина+1)
    
    return Узел(best_split, левое_поддерево, правое_поддерево)
```

---

**5. Дополнительные оптимизации XGBoost**

**Column Subsampling:**
- Случайный выбор подмножества признаков на каждом уровне/дереве
- Уменьшает переобучение, ускоряет обучение
- Параметры: `colsample_bytree`, `colsample_bylevel`, `colsample_bynode`

**Row Subsampling:**
- Случайный выбор подмножества примеров для каждого дерева
- Параметр: `subsample` (обычно 0.5-1.0)

**Approximate Split Finding:**
- Вместо перебора всех порогов, используем квантили
- Параметр: `tree_method='approx'` или `'hist'`

**Weighted Quantile Sketch:**
- Квантили взвешиваются по hessian (больший вес = важнее)
- Учитывает неравномерное распределение примеров

**Sparsity-Aware Split Finding:**
- Эффективная обработка missing values и разреженных данных
- Автоматически учится "куда отправлять missing values" (влево или вправо)

**Cache-Aware Access:**
- Данные хранятся в блоках, оптимизированных для CPU cache
- Значительно ускоряет обучение

**Out-of-Core Computation:**
- Может работать с данными, не помещающимися в RAM
- Параметр: `tree_method='external'`

---

### 1.6 Ключевые гиперпараметры XGBoost

#### Параметры дерева (Tree Parameters)

| Параметр | Описание | Типичные значения | Влияние |
|----------|----------|-------------------|----------|
| `max_depth` | Максимальная глубина дерева | 3-10 | ↑ глубина → ↑ сложность, ↑ overfitting |
| `min_child_weight` | Минимальная сумма hessian в листе | 1-10 | ↑ вес → более консервативные разбиения |
| `gamma` | Минимальный gain для разбиения | 0-5 | ↑ gamma → меньше разбиений, ↓ overfitting |
| `subsample` | Доля примеров для каждого дерева | 0.5-1.0 | < 1 → ↓ overfitting, но ↑ дисперсия |
| `colsample_bytree` | Доля признаков для каждого дерева | 0.3-1.0 | < 1 → ↓ overfitting, ↑ разнообразие |
| `colsample_bylevel` | Доля признаков для каждого уровня | 0.3-1.0 | Дополнительная регуляризация |

#### Параметры регуляризации

| Параметр | Описание | Типичные значения |
|----------|----------|-------------------|
| `lambda` (L2) | $\lambda$ в формуле регуляризации | 0-10 |
| `alpha` (L1) | L1 регуляризация на веса | 0-10 |
| `max_delta_step` | Ограничение на шаг обновления | 0-10 (0 = нет ограничения) |

#### Параметры обучения

| Параметр | Описание | Типичные значения | Влияние |
|----------|----------|-------------------|----------|
| `learning_rate` (η) | Скорость обучения (shrinkage) | 0.01-0.3 | ↓ lr → нужно больше деревьев, но лучше качество |
| `n_estimators` | Количество деревьев | 100-10000 | ↑ деревья + ↓ lr = лучшее качество |
| `early_stopping_rounds` | Остановка если нет улучшения | 10-50 | Предотвращает overfitting |

#### Стратегии тюнинга

**Этап 1: Фиксируем высокий learning rate (0.1)**
- Тюним: `max_depth`, `min_child_weight`
- Цель: Найти правильную сложность дерева

**Этап 2: Добавляем sampling**
- Тюним: `subsample`, `colsample_bytree`
- Цель: Увеличить разнообразие деревьев

**Этап 3: Регуляризация**
- Тюним: `gamma`, `lambda`, `alpha`
- Цель: Fine-tune борьбы с overfitting

**Этап 4: Снижаем learning rate**
- Уменьшаем `learning_rate` до 0.01-0.05
- Увеличиваем `n_estimators` пропорционально
- Используем `early_stopping_rounds`
- Цель: Финальная оптимизация качества

---

### 1.7 Интерпретация Feature Importance

XGBoost предоставляет 3 типа feature importance:

#### 1. Weight (Frequency)

$$\text{Importance}_{weight}(k) = \frac{\text{число разбиений по признаку } k}{\text{общее число разбиений}}$$

**Когда использовать:** Быстрая оценка, какие признаки часто используются.

**Недостатки:** Не учитывает, насколько полезны разбиения.

#### 2. Gain (Average Information Gain)

$$\text{Importance}_{gain}(k) = \sum_{\text{splits by } k} \text{Gain}$$

Средний gain от всех разбиений по признаку $k$.

**Когда использовать:** Показывает, какие признаки дают наибольшее улучшение loss.

**Рекомендуется по умолчанию!**

#### 3. Cover (Sum of Hessian)

$$\text{Importance}_{cover}(k) = \sum_{\text{splits by } k} H_L + H_R$$

Сумма вторых градиентов (hessian) для всех разбиений по признаку.

**Интуиция:** Hessian = "количество примеров × их важность". Признаки, покрывающие важные примеры, получают высокий cover.

**Когда использовать:** Учитывает не только улучшение, но и сколько примеров затронуто.

---

### 1.8 XGBoost vs другие методы

| Аспект | Random Forest | Gradient Boosting | XGBoost | LightGBM | CatBoost |
|--------|---------------|-------------------|---------|----------|----------|
| **Стратегия** | Параллельные независимые деревья | Последовательные деревья | GBM + regularization | Leaf-wise growth | Ordered boosting |
| **Скорость** | Быстро | Медленно | Быстро | Очень быстро | Средне |
| **Точность** | Хорошо | Очень хорошо | Отлично | Отлично | Отлично |
| **Overfitting** | Устойчив | Склонен | Средне (из-за регуляризации) | Склонен | Устойчив |
| **Категории** | One-hot | One-hot | One-hot | Native support | Native support |
| **Missing values** | Нет | Нет | Автоматически | Автоматически | Автоматически |
| **Интерпретация** | Средне | Хорошо | Хорошо | Хорошо | Хорошо |

**Когда использовать XGBoost:**
- ✅ Нужна максимальная точность на табличных данных
- ✅ Есть время на тюнинг гиперпараметров
- ✅ Датасет среднего размера (10k-1M примеров)
- ✅ Важна интерпретируемость
- ❌ Нужна очень быстрая инференция (используйте Random Forest)
- ❌ Очень большой датасет >10M примеров (используйте LightGBM)
- ❌ Много категориальных признаков (используйте CatBoost)

---

## Теоретическая часть завершена! Переходим к практике 🚀

## 📊 Часть 2: Загрузка данных и предварительный анализ

### 2.1 Импорт библиотек

In [ ]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve
)

# Baseline модели для сравнения
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Seed для воспроизводимости
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Библиотеки загружены')
print(f'XGBoost version: {xgb.__version__}')